# Core

> The Plash CLI tool

In [ ]:
#| default_exp core

In [ ]:
#| export
from fastcore.all import *
from fastcore.xdg import *
import secrets, webbrowser, json, httpx, io, tarfile
from pathlib import Path
from uuid import uuid4
from time import time, sleep

import io, sys, tarfile

## Helpers

In [ ]:
#| export
PLASH_CONFIG_HOME = xdg_config_home() / 'plash_config.json'

In [ ]:
#| export
def get_client(cookie_file):
    client = httpx.Client()
    if not cookie_file.exists():
        raise FileNotFoundError("Plash config not found. Please run plash_login and try again.")
    cookies = Path(cookie_file).read_json()
    client.cookies.update(cookies)
    return client

In [ ]:
#| export
def get_app_id(path:Path):
    plash_app = Path(path) / '.plash'
    if not plash_app.exists(): raise FileNotFoundError(f"File not found: {plash_app=}")
    return parse_env(fn=plash_app)['PLASH_APP_ID']

In [ ]:
#| export
def endpoint(path, local, port=None):
    p = "http" if local else "https"
    d = f"localhost:{port}" if local else "pla.sh"
    return f"{p}://{d}{path}"

In [ ]:
#| export
def is_included(path):
    "Returns True if path should be included in deployment"
    if path.name.startswith('.'): return False
    if path.suffix == '.pyc': return False
    excludes = {'.git', '__pycache__', '.gitignore', '.env', 
                '.pytest_cache', '.venv', 'venv', '.ipynb_checkpoints',
                '.vscode', '.idea', '.sesskey'}
    return not any(p in excludes for p in path.parts)

In [ ]:
#| export
def create_tar_archive(path # Path to directory containing FastHTML app
                      )->io.BytesIO: # Buffer of tar directory
    "Creates a tar archive of a directory, excluding files based on is_included"
    buf = io.BytesIO()
    files = L(Path(path).iterdir()).filter(is_included)

    with tarfile.open(fileobj=buf, mode='w:gz') as tar:
        for f in files: tar.add(f, arcname=f.name)
    buf.seek(0)
    return buf, len(files)

In [ ]:
#| export
def validate_app(path):
    "Validates that the app in the directory `path` is deployable as a FastHTML app"
    print("Analyzing project structure...")

    main_file = Path(path) / "main.py"
    if not main_file.exists():
        print('[red bold]ERROR: Your FastHTML app must have a main.py[/red bold]')
        print(f'Your path is: [bold]{path}[/bold]')
        sys.exit(1)

## Plash - login

In [ ]:
#| export
def poll_cookies(paircode, local, port=None, interval=1, timeout=180):
    "Poll server for token until received or timeout"
    start = time()
    client = httpx.Client()
    url = endpoint(f"/cli_token?paircode={paircode}",local,port)
    while time()-start < timeout:
        resp = client.get(url).raise_for_status()
        if resp.text.strip(): return dict(client.cookies)
        sleep(interval)
        
@call_parse
def login(
    local:bool=False,  # Local dev
    port:int=5002      # Port for local dev
):
    "Authenticate CLI with server and save config"
    paircode = secrets.token_urlsafe(16)
    login_url = httpx.get(endpoint(f"/cli_login?paircode={paircode}",local,port)).text
    print(f"Opening browser for authentication:\n{login_url}\n")
    webbrowser.open(login_url)
    
    cookies = poll_cookies(paircode, local, port)
    if cookies:
        Path(PLASH_CONFIG_HOME).write_text(json.dumps(cookies))
        print(f"Authentication successful! Config saved to {PLASH_CONFIG_HOME}")
    else: print("Authentication timed out.")

## App - deploy

In [ ]:
#| export
@call_parse
def deploy(
    path:Path=Path('.'), # Path to project
    local:bool=False,    # Local dev
    port:int=5002):      # Port for local dev
    """🚀 Ship your app to production"""
    client = get_client(PLASH_CONFIG_HOME)
    print('Initializing deployment...')
    validate_app(path)
    tarz, filecount = create_tar_archive(path)

    plash_app = Path(path) / '.plash'
    if not plash_app.exists():
        # Create the .plash file and write the app name
        plash_app.write_text(f'export PLASH_APP_ID=fasthtml-app-{str(uuid4())[:8]}')
    aid = parse_env(fn=plash_app)['PLASH_APP_ID']

    resp = client.post(endpoint("/upload",local,port), files={'file': tarz}, timeout=300.0, data={'aid': aid})
    if resp.status_code == 200: 
        print('✅ Upload complete! Your app is currently being built.')
        if local: print(f'It will be live at http://{aid}.localhost')
        else: print(f'It will be live at https://{aid}.pla.sh')
    else:
        print(f'Failure {resp.status_code}')
        print(f'Failure {resp.text}')

## App - view

In [ ]:
#| export
@call_parse
def view(
    path:Path=Path('.'), # Path to project
    local:bool=False,    # Local dev
):
    aid = get_app_id(path)
    if local: url = f"http://{aid}.localhost"
    else: url = f"https://{aid}.pla.sh"
    print(f"Opening browser to view app :\n{url}\n")
    webbrowser.open(url)

## App - delete

In [ ]:
#| export
@call_parse
def delete(
    path:Path=Path('.'), # Path to project
    local:bool=False,    # Local dev
    port:int=5002,       # Port for local dev
    force:bool=False):   # Skip confirmation prompt
    'Delete your deployed app'
    client = get_client(PLASH_CONFIG_HOME)
    aid = get_app_id(path)
    if not force:
        confirm = input(f"Are you sure you want to delete app '{aid}'? This action cannot be undone. [y/N]: ")
        if confirm.lower() not in ['y', 'yes']:
            print("Deletion cancelled.")
            return
    
    print(f"Deleting app '{aid}'...")
    r = client.delete(endpoint(f"/delete?aid={aid}",local,port))
    return r.text

## App - start stop logs

In [ ]:
#| export
def endpoint_func(endpoint_name):
    'Creates a function for a specific API endpoint'
    @call_parse
    def func(
        path:Path=Path('.'), # Path to project
        local:bool=False,    # Local dev
        port:int=5002):      # Port for local dev
        client = get_client(PLASH_CONFIG_HOME)
        aid = get_app_id(path)
        r = client.get(endpoint(f"{endpoint_name}?aid={aid}", local, port))
        return r.text
    
    # Set the function name and docstring
    func.__name__ = endpoint_name
    func.__doc__ = f"Access the '{endpoint_name}' endpoint for your app"
    
    return func

# Create endpoint-specific functions
stop = endpoint_func('/stop')
start = endpoint_func('/start')
logs = endpoint_func('/logs')

## App - download

In [ ]:
#| export
@call_parse
def download(
    path:Path=Path('.'),                # Path to project
    save_path:Path=Path("./download/"), # Save path (optional) 
    local:bool=False,                   # Local dev
    port:int=5002,                      # Port for local dev
    ):
    'Download your deployed app.'
    client = get_client(PLASH_CONFIG_HOME)
    aid = get_app_id(path)
    try: save_path.mkdir(exist_ok=False)
    except: print(f"ERROR: Save path ({save_path}) already exists. Please rename or delete this folder to avoid accidental overwrites.")
    else:
        response = client.get(endpoint(f'/download?aid={aid}', local, port)).raise_for_status()
        file_bytes = io.BytesIO(response.content)
        with tarfile.open(fileobj=file_bytes, mode="r:gz") as tar: tar.extractall(path=save_path)
        print(f"Downloaded your app to: {save_path}")

## Export

In [ ]:
#|hide
#|eval: false
from nbdev.doclinks import nbdev_export
nbdev_export()

## Current Readme

# plash-cli


<!-- WARNING: THIS FILE WAS AUTOGENERATED! DO NOT EDIT! -->

**WARNING** - Plash is in Beta and we have released it in its
semi-stable state to gather early feedback to improve. We do not
recommend hosting critical applications yet.

## Usage

### Installation

Install latest from the GitHub
[repository](https://github.com/AnswerDotAI/plash-cli):

``` sh
$ pip install git+https://github.com/AnswerDotAI/plash-cli.git
```

or from [pypi](https://pypi.org/project/plash-cli/)

``` sh
$ pip install plash-cli
```

## Deploy Your First FastHTML App

### Set Up Your Plash Token

You’ll need to set a PLASH_TOKEN and PLASH_EMAIL environment variables
to use Plash. To obtain these, do the following:

1.  Signup for an account at https://pla.sh/
2.  Activate your Plash subscription
3.  Follow instructions in your Plash dashboard to save credentials
    locally

### Create a FastHTML App

Create a directory for your FastHTML app, and go into it:

``` sh
mkdir minimal
cd minimal
```

Create `main.py` containing:

``` python
from fasthtml.common import *

app, rt = fast_app()

@rt
def index():
    return H1("Hello world!")

serve()
```

Then create a requirements.txt containing:

    python-fasthtml

### Deploy your app

Run `plash_deploy`. Your app will be live at
`https://<app-name>.pla.sh`. The URL will be shown in the deployment
output.

### App Dependencies

If your app needs additional dependencies to run, we offer a number of
ways to have them included in your deployed app.

#### Python Dependencies

To have python dependencies installed in your deployed app, you can
provide a `requirements.txt` and it will be pip installed. By default,
all deployed apps have fasthtml as a dependency.

#### Non-Python Dependencies

For any other depencies of setup processes, you can provide a `setup.sh`
which will be executed during the build step of your app. For example,
you can use this to install apt packages (this is ran as root in your
apps container, so omit any `sudo`):

``` bash
#!/bin/bash
apt install <package_name>
```

#### Env Variables

If your app depends on secrets or other types of environment variables,
you can have them available in your deployed app by providing a
`plash.env`, which will be sourced during your apps startup. Here is an
example:

    export MY_ENV_VARIABLE=hijkl
    export ANOTHER_SECRET=abcdef

Inside of your running container, we automatically set an environment
variable (`PLASH_PRODUCTION=1`) so you are able to use it for checking
if your application is inside a Plash deployment or not.

## Databases

For apps that use persistent storage, we recommend sqlite. The docker
container your app runs in has a working directory of /app which is a
volume mounted to a folder that we hourly backup. Therefore, we
recommend placing your sqlite database somewhere in that directory. Note
when redeploying an app with plash_deploy, we automatically overwrite
existing files with the same name as those uploaded. Therefore to
prevent data loss, ensure any local database files do not clash with any
deployed database names that your app may set up. You can use the
environment variable `PLASH_PRODUCTION`, which we automatically set to 1
in your Plash container, to modify your apps behavior for local and
production development. You can download any deployed database names by
clicking the Download App button to get a compressed file of all files
in your /app folder in your deployed app.

## Deploy to Pla.sh via GitHub Actions

If you’d like to deploy your plash app every time you commit changes to
your GitHub repo, you can use the following workflow to your
`.github/workflows/` folder in the root of your repo:

``` yaml
yaml 
name: Deploy to Plash

on:
  push:
    branches:
      - main
  workflow_dispatch:

jobs:
  deploy:
    runs-on: ubuntu-latest
    steps:
      - name: Checkout repository
        uses: actions/checkout@v3

      - name: Set up Python
        uses: actions/setup-python@v4
        with:
          python-version: '3.x'

      - name: Create Plash config
        run: |
          mkdir -p ~/.config
          echo "PLASH_EMAIL=${{ secrets.PLASH_EMAIL }}" > ~/.config/plash.env
          echo "PLASH_TOKEN=${{ secrets.PLASH_TOKEN }}" >> ~/.config/plash.env

      - name: Install plash_cli with pip
        run: pip install plash_cli

      - name: Deploy to Plash
        run: plash_deploy
```

It relies on storing your `PLASH_EMAIL` and `PLASH_TOKEN` as secrets in
your GitHub repo, which you can find more information on how to do that
[here](https://docs.github.com/en/actions/security-for-github-actions/security-guides/using-secrets-in-github-actions).


## New Readme

In [ ]:
%%ai

The Current Readme is the old readme. I've changed authentication, and added a bunch of new cli commands.
Please rewrite the readme to be up to date w the changes.

# plash-cli

<!-- WARNING: THIS FILE WAS AUTOGENERATED! DO NOT EDIT! -->

**WARNING** - Plash is in Beta and we have released it in its
semi-stable state to gather early feedback to improve. We do not
recommend hosting critical applications yet.

## Usage

### Installation

Install latest from the GitHub
[repository](https://github.com/AnswerDotAI/plash-cli):

```sh
$ pip install git+https://github.com/AnswerDotAI/plash-cli.git
```

or from [pypi](https://pypi.org/project/plash-cli/)

```sh
$ pip install plash-cli
```

## Deploy Your First FastHTML App

### Authentication

To use Plash, you'll need to authenticate:

1. Signup for an account at https://pla.sh/
2. Activate your Plash subscription
3. Run `plash_login` in your terminal
4. A browser window will open for authentication
5. Once authenticated, your credentials will be saved locally

### Create a FastHTML App

Create a directory for your FastHTML app, and go into it:

```sh
mkdir minimal
cd minimal
```

Create `main.py` containing:

```python
from fasthtml.common import *

app, rt = fast_app()

@rt
def index():
    return H1("Hello world!")

serve()
```

Then create a requirements.txt containing:

    python-fasthtml

### Deploy your app

Run `plash_deploy`. Your app will be live at
`https://<app-id>.pla.sh`. The URL will be shown in the deployment
output.

### Managing Your App

Plash CLI provides several commands to manage your deployed app:

- `plash_view` - Open your deployed app in a browser
- `plash_start` - Start your app if it's stopped
- `plash_stop` - Stop your running app
- `plash_logs` - View your app's logs
- `plash_download` - Download your deployed app files
- `plash_delete` - Delete your deployed app

### App Dependencies

If your app needs additional dependencies to run, we offer a number of
ways to have them included in your deployed app.

#### Python Dependencies

To have python dependencies installed in your deployed app, you can
provide a `requirements.txt` and it will be pip installed. By default,
all deployed apps have fasthtml as a dependency.

#### Non-Python Dependencies

For any other dependencies or setup processes, you can provide a `setup.sh`
which will be executed during the build step of your app. For example,
you can use this to install apt packages (this is ran as root in your
apps container, so omit any `sudo`):

```bash
#!/bin/bash
apt install <package_name>
```

#### Env Variables

If your app depends on secrets or other types of environment variables,
you can have them available in your deployed app by providing a
`plash.env`, which will be sourced during your apps startup. Here is an
example:

    export MY_ENV_VARIABLE=hijkl
    export ANOTHER_SECRET=abcdef

Inside of your running container, we automatically set an environment
variable (`PLASH_PRODUCTION=1`) so you are able to use it for checking
if your application is inside a Plash deployment or not.

## Databases

For apps that use persistent storage, we recommend sqlite. The docker
container your app runs in has a working directory of /app which is a
volume mounted to a folder that we hourly backup. Therefore, we
recommend placing your sqlite database somewhere in that directory. Note
when redeploying an app with plash_deploy, we automatically overwrite
existing files with the same name as those uploaded. Therefore to
prevent data loss, ensure any local database files do not clash with any
deployed database names that your app may set up. You can use the
environment variable `PLASH_PRODUCTION`, which we automatically set to 1
in your Plash container, to modify your apps behavior for local and
production development. You can download any deployed database names by
using the `plash_download` command to get a compressed file of all files
in your /app folder in your deployed app.

## Deploy to Pla.sh via GitHub Actions

If you'd like to deploy your plash app every time you commit changes to
your GitHub repo, you can use the following workflow to your
`.github/workflows/` folder in the root of your repo:

```yaml
name: Deploy to Plash

on:
  push:
    branches:
      - main
  workflow_dispatch:

jobs:
  deploy:
    runs-on: ubuntu-latest
    steps:
      - name: Checkout repository
        uses: actions/checkout@v3

      - name: Set up Python
        uses: actions/setup-python@v4
        with:
          python-version: '3.x'

      - name: Install plash_cli
        run: pip install plash-cli

      - name: Login to Plash
        run: |
          mkdir -p ~/.config
          echo '${{ secrets.PLASH_COOKIES }}' > ~/.config/plash_config.json

      - name: Deploy to Plash
        run: plash_deploy
```

It relies on storing your authentication cookies as a secret named `PLASH_COOKIES` in
your GitHub repo. You can find these in `~/.config/plash_config.json` after running `plash_login`.
Learn more about GitHub secrets [here](https://docs.github.com/en/actions/security-for-github-actions/security-guides/using-secrets-in-github-actions).

In [ ]:
%%ai

Which sections were changed? just give me the headers

The following sections were changed:

### Authentication
### Managing Your App
### Deploy to Pla.sh via GitHub Actions